In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# Load files and create dataframes in pandas

#2015
happiness_data_2015 = "Resources/2015_Happiness_data.csv"
happiness_df_2015 = pd.read_csv(happiness_data_2015)
# Set Index to Country, rename columns and drop unnecessary
happiness_df_2015 = happiness_df_2015.set_index("Country").rename(columns = {
    "Happiness Rank":"2015_rank", "Happiness Score": "2015_score", "Economy (GDP per Capita)": "2015_GDP", 
    "Family": "2015_family","Health (Life Expectancy)": "2015_life_expect", "Freedom": "2015_freedom", 
    "Generosity": "2015_generosity", "Trust (Government Corruption)": "2015_corruption", 
    "Dystopia Residual": "2015_Dystopia"}).drop(["Region", "Standard Error"], axis = 1)
# happiness_df_2015.to_csv("resources/Happiness_data_clean_2015.csv")

#2016
happiness_data_2016 = "Resources/2016_Happiness_data.csv"
happiness_df_2016 = pd.read_csv(happiness_data_2016)
# Set Index to Country
happiness_df_2016 = happiness_df_2016.set_index("Country")
# Rename columns and drop unnecessary
happiness_df_2016 = happiness_df_2016.rename(columns = {
    "Happiness Rank":"2016_rank", "Happiness Score": "2016_score", "Economy (GDP per Capita)": "2016_GDP", 
    "Family": "2016_family","Health (Life Expectancy)": "2016_life_expect", "Freedom": "2016_freedom", 
    "Generosity": "2016_generosity", "Trust (Government Corruption)": "2016_corruption", 
    "Dystopia Residual": "2016_Dystopia"}).drop(["Region", "Lower Confidence Interval", "Upper Confidence Interval"], axis = 1)
# happiness_df_2016.to_csv("resources/Happiness_data_clean_2016.csv")

#2017
happiness_data_2017 = "Resources/2017_Happiness_data.csv"
happiness_df_2017 = pd.read_csv(happiness_data_2017)
# Set Index to Country, rename columns and drop unnecessary
happiness_df_2017 = happiness_df_2017.set_index("Country").rename(columns = {
    "Happiness.Rank":"2017_rank", "Happiness.Score": "2017_score", "Economy..GDP.per.Capita.": "2017_GDP", 
    "Family": "2017_family","Health..Life.Expectancy.": "2017_life_expect", "Freedom": "2017_freedom", 
    "Generosity": "2017_generosity", "Trust..Government.Corruption.": "2017_corruption", 
    "Dystopia.Residual": "2017_Dystopia"}).drop(["Whisker.high", "Whisker.low"], axis = 1)
# happiness_df_2017.to_csv("resources/Happiness_data_clean_2017.csv")

happiness_df_2017.head()


NameError: name 'pd' is not defined

In [3]:
# Merge Data Frames to give summary data frame across time

merged_1516_df = pd.merge(happiness_df_2015, happiness_df_2016, how="left", on=["Country"])

happiness_summary_df = pd.merge(merged_1516_df, happiness_df_2017, how ="left", on=["Country"])

happiness_summary_df.head()


NameError: name 'pd' is not defined

In [4]:
# Create Summary Dataframe
average_rank = []
average_score = []
average_GDP
average_family = []
average_life_expect = []
average_freedom = []
average_corruption = []
average_generosity = []
happiness_rank = [unhappy, neutral, moderately happy, extremely happy]


summary_df = 

# For loop

SyntaxError: invalid syntax (<ipython-input-4-2a3f483b6077>, line 10)

In [5]:
# Happiness Data Analysis
# Function for scatter plot and linear regression
def regression(x_axis, y_axis, title, x_label, y_label, chart_path):
    slope, y_int, r, pval, stderr = linregress(x_axis, y_axis)
    plt.scatter(x_axis, y_axis)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.grid(b=True, which="major",axis="both", color="lightgrey")
    regression = slope * x_axis + y_int
    plt.plot(x_axis, regression, color = "red")
    plt.show()

# GDP vs. happiness
plt.subplot(2, 2, 1)
# regression(northern_df["Lat"], northern_df["Temp"], "Northern Hemisphere- Temp vs. Latitude", 
           "Latitude", "Temperature (F)", "Images/Plot5.png" )

# t-test

# Freedom vs. happiness

# Generosity vs happiness

# Family vs happiness

# Corruption vs. happiness


IndentationError: unexpected indent (<ipython-input-5-215319ca6f09>, line 17)

In [6]:
# Group by for income brackets (4)- add to summary dataframe

In [7]:
# Box and Whiskers- happiness by income bracket

In [8]:
# create bins for corruption, create bins (4 each) for happiness add to summary df

In [9]:
# Heat map- happiness by corruption 